# **Required Libraries**

In [1]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# **Loading Data**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DebertaForSequenceClassification, DebertaTokenizer
from datasets import Dataset

# Load the data again and check the structure
data = pd.read_excel('/content/Reviews Classification_output.xlsx')

# Mapping labels to integers
label_map = {
    "Positive Feedback": 0,
    "Product Recommendations": 0,
    "Product Quality Issue": 1,
    "Authenticity and Originality Concerns": 1,
    "Delivery and Logistics Issues": 1,
    "Customer Service Experience": 1,
    "Returns, Refunds, and Exchange Issues": 1,
    "Scam Complaint": 1,
    "Pricing and Billing Issues": 1,
    "Warranty and Support Complaints": 1,
}

# Create a new column for numerical labels
data['label'] = data['Review_CAT'].map(label_map)

# Split data into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Convert to Hugging Face dataset
train_dataset = Dataset.from_pandas(train_data[['description', 'label']])
val_dataset = Dataset.from_pandas(val_data[['description', 'label']])


# **Tokenizing Data**

In [3]:
from transformers import DebertaTokenizer

# Initialize the tokenizer
tokenizer = DebertaTokenizer.from_pretrained("microsoft/deberta-base")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['description'], padding=True, truncation=True)

# Apply tokenization to datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Ensure the correct format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/120 [00:00<?, ? examples/s]

# **Initialize the Model**

In [ ]:
import torch
from transformers import DebertaForSequenceClassification, Trainer, TrainingArguments

# Force the default tensor type to float32
torch.set_default_dtype(torch.float32)

# Load DeBERTa-base model with num_labels argument
model = DebertaForSequenceClassification.from_pretrained("microsoft/deberta-base", num_labels=2)

# Set up training arguments for CPU with explicit FP32
training_args = TrainingArguments(
    output_dir='./results',  # Output directory for model and checkpoints
    evaluation_strategy="steps",  # Evaluate after every N steps
    save_strategy="steps",  # Save model every N steps
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=1,  # Batch size for training
    per_device_eval_batch_size=1,  # Batch size for evaluation
    num_train_epochs=3,  # Number of epochs
    weight_decay=0.01,  # Regularization
    logging_dir='./logs',  # Directory for logs
    logging_steps=100,  # Log every 100 steps
    save_steps=200,  # Save checkpoint every 200 steps
    load_best_model_at_end=True,  # Load the best model based on evaluation
    no_cuda=True,  # Force training on CPU
    fp16=False,  # Disable mixed precision
    bf16=False,  # Disable BFloat16 precision
    report_to="none"  # Disable WANDB if you don't want it
)

# Ensure model's data type is float32
model.to(torch.float32)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model on CPU
trainer.train()


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
100,0.996500,1.335884


Step,Training Loss,Validation Loss
100,0.996500,1.335884
200,1.356400,1.186346
300,1.475500,1.152564
